In [2]:
import json

import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
import sys
import os

# Get the current working directory and the relative path to your module
module_path = os.path.abspath(os.path.join("..", ".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from src.api.service.repo import SearchRepo

In [4]:
es = SearchRepo()

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '9I9q3mFYSISbvBg75cK23A',
 'name': '5eeda383872f',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


# 1. Test Ingest Pipeline

In [ ]:
processors = [
    {
      "script": {
        "description": "Expand termFacetData: split list values into separate entries while keeping scalar values intact.",
        "lang": "painless",
        "source": """
          def expanded = [];
          for (entry in ctx.termFacetData) {
            if (entry.value instanceof List) {
              for (item in entry.value) {
                expanded.add(['name': entry.name, 'value': item]);
              }
            } else {
              expanded.add(entry);
            }
          }
          ctx.termFacetData = expanded;
        """
      }
    },
    # {
    #   "script": {
    #     "description": "Decompose hierarchical pathFacetData into multiple levels, preserving original and adding parent paths.",
    #     "lang": "painless",
    #     "source": """
    #       def newPathFacetData = []; // Temporary list to store results
    #       for (entry in ctx.pathFacetData) {
    #         def path = entry.value.splitOnToken('/');
    #         def accumulated = [];
    #         for (segment in path) {
    #           accumulated.add(segment);
    #           newPathFacetData.add(['name': entry.name, 'value': accumulated.join('/')]);
    #         }
    #       }
    #       ctx.pathFacetData = newPathFacetData; // Replace the original list
    #     """
    #   }
    # }
  ]

In [ ]:
resp = es.es.ingest.put_pipeline(
    id="ingest",
    description="This pipeline processes `termFacetData` and `pathFacetData` fields to prepare them for indexing. For `termFacetData`, it expands list values into separate entries while retaining scalar values as-is. For `pathFacetData`, it decomposes hierarchical paths into multiple levels, preserving the full path and adding its parent paths. This ensures both fields are normalized for efficient querying and analysis.",
    processors=processors,
)
resp.body

In [ ]:
document={
    "termFacetData": [
        {"name": "year", "value": "2008"},
        {"name": "engine_type", "value": "I4"},
        {"name": "engine_size", "value": "1.8L"},
        {"name": "transmission", "value": "Manual"},
        {"name": "drive_type", "value": "RWD"},
        {"name": "fuel_type", "value": "Gasoline"},
        {"name": "exterior_color", "value": "Red"},
        {"name": "interior_color", "value": "Standard"},
        {"name": "trim_level", "value": "Standard"},
        {
            "name": "notable_features",
            "value": ["Leather seats", "Navigation", "Bluetooth"],
        },
        {"name": "safety_features", "value": ["ABS", "Airbags", "Stability control"]},
    ]
}

In [ ]:
doc = [{"_source":document}]

In [ ]:
resp = es.es.ingest.simulate(id="ingest", docs=doc)
resp.body